In [2]:
import pandas as pd
import numpy as np
#import extraction_squares
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os 
import sys

operating_system = 'mac'

if operating_system == 'win':
    os.chdir('C:/Users/fabau/OneDrive/Documents/GitHub/master-project-cleaned/')
elif operating_system == 'curnagl':
    os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')
    path = os.getcwd()
else:
    os.chdir('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/')

# Add the path to the custom library
custom_library_path = os.path.abspath('util/processing/')
sys.path.append(custom_library_path)

import extraction_squares, time_series#, pca

'''if operating_system == 'curnagl':
    name_of_variable= pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS/variable_list_80_mean.csv')
    path_data = '/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/DATASETS'
else:'''
variables= pd.read_csv('data/variable_list_levels.csv')
path_data = 'data'
path = os.getcwd()

'''storm_dates = pd.read_csv('pre_processing/tracks/storm_dates.csv')
path_tracks_1h_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_EU'
path_tracks_1h_non_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_non_EU'
dataset = 'datasets_1h_EU'
dataset_non_EU = 'datasets_1h_non_EU'''

"storm_dates = pd.read_csv('pre_processing/tracks/storm_dates.csv')\npath_tracks_1h_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_EU'\npath_tracks_1h_non_EU = 'pre_processing/tracks/ALL_TRACKS/tracks_1h_non_EU'\ndataset = 'datasets_1h_EU'\ndataset_non_EU = 'datasets_1h_non_EU"

In [8]:
# Data is stored by statistics
stats = ['max', 'min', 'mean', 'std']

# Initialize empty DataFrames for the final combined sets
final_train = pd.DataFrame()
final_test = pd.DataFrame()
final_validation = pd.DataFrame()

all_scores = pd.DataFrame()
all_loadings = pd.DataFrame()

all_time_series = pd.DataFrame()

# Load the data
for var in variables['variables']:
    if var == 'sea_surface_temperature':
        print('sea_surface_temperature is not taken into account')
        continue

    if var == 'instantaneous_10m_wind_gust':
        print('instantaneous_10m_wind_gust is not taken into account, for obvious reasons')
        continue

    if var.startswith('specific_rain_water_'):
        print('specific_rain_water_content is not taken into account, because it is just 0')
        continue
    if var.startswith('convective'):
        print('convective_precipitation is not taken into account, because it is just 0')
        continue

    # Process each statistic for the current variable
    for stat in stats:
        var_name = f"{var}_{stat}"
        print(f"Processing {var_name}...")
        var_data = pd.read_csv(f"{path}/data/time_series_1h_non_EU/{var}/{var}_{stat}.csv")
        test_var_data = var_data.drop(columns=['Unnamed: 0'])

        time_series_df = test_var_data.copy()
        #time_series_df['variable'] = var_name
        time_series_df = time_series_df.drop(columns=['storm_index']).to_numpy()
        time_series_df = time_series_df = pd.DataFrame(
                                                time_series_df, 
                                                columns=[f'{var_name}_step_{i}' for i in range(36)])
        #time_series_df['storm_index'] = var_data['storm_index']
        #time_series_df = time_series_df.T
        #time_series_df['variable'] = var_name
        #time_series_df = time_series_df.T


        # Threshold for variance retention
        threshold = 0.8  # 98% variance
        seed_number = 42

        # Assume var_all_scaler has columns: storm_number, step_1, step_2, ..., step_n

        # Step 1: Extract numerical data (exclude storm_number for scaling)
        storm_numbers = test_var_data['storm_index']
        data_to_scale = test_var_data.drop(columns=['storm_index'])

        # TO DISCUSS, standardize by storms per variable or just by variable ?

        # Step 2: Scale the entire dataset
        scaler = StandardScaler()
        #data_flattened = data_to_scale.values.flatten().reshape(-1, 1)
        data_scaled_flattened = scaler.fit_transform(data_to_scale)
        #data_scaled = data_scaled_flattened.reshape(data_to_scale.shape)
        data_scaled = data_scaled_flattened

        # Convert scaled data back to DataFrame with storm_number
        data_scaled_df = pd.DataFrame(data_scaled)#, columns=data_to_scale.columns)
        #data_scaled_df['storm_index'] = storm_numbers.values

        # Transpose the data
        data_scaled = data_scaled_df.T  # Shape becomes (time_steps, storms)

        # Apply PCA
        pca = PCA(n_components=threshold, random_state=seed_number)
        data_pca = pca.fit_transform(data_scaled)

        # Calculate cumulative explained variance
        explained_variance = pca.explained_variance_ratio_.cumsum()

        # Find the number of components to explain 'threshold' variance
        #n_components_idx = np.argmax(explained_variance >= threshold) + 1  # +1 to convert to count
        n_components = len(explained_variance)
        print(f"To retain {threshold*100}% of the variance, we need {n_components} components.")

        # Transform the data to the reduced number of components
        #data_pca = pca.transform(data_scaled)[:, :n_components_idx]

        # Convert back to DataFrame
        data_pca_df = pd.DataFrame(data_pca, columns=[f'PCA_{i+1}' for i in range(n_components)])
        #data_pca_df = pd.DataFrame(data_pca)

        # Add storm_number back for reference if needed
        #data_pca_df['storm_number'] = storm_numbers.values
        data_pca_df['time_step'] = range(1, data_pca_df.shape[0] + 1)  # Label time steps instead

        # put the storm number in the first column
        cols = data_pca_df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        data_pca_df = data_pca_df[cols]

        #cols_ts = time_series_df.columns.tolist()
        #cols_ts = cols_ts[-1:] + cols_ts[:-1]
        #time_series_df = time_series_df[cols_ts]

        # add the variable name and stat to the data
        data_pca_df['variable'] = var_name

        # and set it as the second column
        cols = data_pca_df.columns.tolist()
        cols = cols[:1] + cols[-1:] + cols[1:-1]
        data_pca_df = data_pca_df[cols]
        
        # Append to results
        #all_storms_pca.append(storm_pca_df)

        # Step 4: Combine results into a single DataFrame
        #final_pca_df = pd.concat(all_storms_pca, axis=0)
        
        data_pca_df.to_csv(f'{path}/data/PCA_scores/{var_name}.csv', index=False)
        all_scores = pd.concat([all_scores, data_pca_df], axis=0)
        all_time_series = pd.concat([all_time_series, time_series_df], axis=1)

        # Save the PCA loadings
        loadings = pd.DataFrame(pca.components_, columns=storm_numbers)
        # add the variable name and stat to the data
        loadings['variable'] = var_name
        # and set it as the first column
        cols = loadings.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        loadings = loadings[cols]
        loadings.to_csv(f'{path}/data/PCA_loadings/{var_name}.csv', index=False)
        all_loadings = pd.concat([all_loadings, loadings], axis=0)

# Save the final combined datasets
all_scores.to_csv(r'data/PCA_scores/all_scores.csv', index=False)
all_scores.to_csv(r'pre_processing/nestedMLR/all_scores.csv', index=False)
all_loadings.to_csv(r'data/PCA_loadings/all_loadings.csv', index=False)
all_loadings.to_csv(r'pre_processing/nestedMLR/all_loadings.csv', index=False)

all_time_series['storm_index'] = var_data['storm_index']
cols_ts = all_time_series.columns.tolist()
cols_ts = cols_ts[-1:] + cols_ts[:-1]
all_time_series = all_time_series[cols_ts]
all_time_series.to_csv(r'data/time_series_1h_non_EU/all_time_series.csv', index=False)

Processing 10m_u_component_of_wind_max...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_u_component_of_wind_min...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_u_component_of_wind_mean...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_u_component_of_wind_std...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_v_component_of_wind_max...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_v_component_of_wind_min...
To retain 80.0% of the variance, we need 3 components.
Processing 10m_v_component_of_wind_mean...
To retain 80.0% of the variance, we need 2 components.
Processing 10m_v_component_of_wind_std...
To retain 80.0% of the variance, we need 2 components.
Processing 2m_dewpoint_temperature_max...
To retain 80.0% of the variance, we need 2 components.
Processing 2m_dewpoint_temperature_min...
To retain 80.0% of the variance, we need 3 components.
Processing 2m_dewpoint_tempe